In [1]:
from scipy.linalg import svd
import numpy as np
import logging

from algora.base import BaseEstimator

np.random.seed(1000)


class PCA(BaseEstimator):
    y_required = False

    def __init__(self, n_components, solver='svd'):
        """Principal component analysis (PCA) implementation.

        Transforms a dataset of possibly correlated values into n linearly
        uncorrelated components. The components are ordered such that the first
        has the largest possible variance and each following component as the
        largest possible variance given the previous components. This causes
        the early components to contain most of the variability in the dataset.

        Parameters
        ----------
        n_components : int
        solver : str, default 'svd'
            {'svd', 'eigen'}
        """
        self.solver = solver
        self.n_components = n_components
        self.components = None
        self.mean = None

    def fit(self, X, y=None):
        self.mean = np.mean(X, axis=0)
        self._decompose(X)

    def _decompose(self, X):
        # Mean centering
        X = X.copy()
        X -= self.mean

        if self.solver == 'svd':
            _, s, Vh = svd(X, full_matrices=True)
        elif self.solver == 'eigen':
            s, Vh = np.linalg.eig(np.cov(X.T))
            Vh = Vh.T

        s_squared = s ** 2
        variance_ratio = s_squared / (s_squared).sum()
        logging.info('Explained variance ratio: %s' % (variance_ratio[0:self.n_components]))
        self.components = Vh[0:self.n_components]

    def transform(self, X):
        X = X.copy()
        X -= self.mean
        return np.dot(X, self.components.T)

    def _predict(self, X=None):
        return self.transform(X)


In [2]:
try:
    from sklearn.model_selection import train_test_split
except ImportError:
    from sklearn.cross_validation import train_test_split
from sklearn.datasets import make_classification

from algora.linear_models import LogisticRegression
from algora.metrics import accuracy
from algora.pca import PCA

# logging.basicConfig(level=logging.DEBUG)

# Generate a random binary classification problem.
X, y = make_classification(n_samples=1000, n_features=100, n_informative=75,
                           random_state=1111, n_classes=2, class_sep=2.5, )


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                        random_state=1111)

for s in ['svd', 'eigen']:
    p = PCA(15, solver=s)

    # fit PCA with training data, not entire dataset
    p.fit(X_train)
    X_train_reduced = p.transform(X_train)
    X_test_reduced = p.transform(X_test)
    
    model = LogisticRegression(lr=0.001, max_iters=2500)
    model.fit(X_train_reduced, y_train)
    predictions = model.predict(X_test_reduced)
    print('Classification accuracy for %s PCA: %s'
          % (s, accuracy(y_test, predictions)))


/Users/siddharth/miniconda3/lib/python3.6/site-packages/autograd/tracer.py:48: RuntimeWarning: overflow encountered in cosh
  return f_raw(*args, **kwargs)
/Users/siddharth/miniconda3/lib/python3.6/site-packages/autograd/numpy/numpy_vjps.py:88: RuntimeWarning: overflow encountered in square
  defvjp(anp.tanh,   lambda ans, x : lambda g: g / anp.cosh(x) **2)


Classification accuracy for svd PCA: 0.92
Classification accuracy for eigen PCA: 0.912
